In [ ]:
from datetime import datetime
from dateutil import parser
import glob
import io
import os
import pdb
from pprint import pprint
import pytz
import re
import time

import chess.pgn
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tqdm import trange

In [ ]:
# This is where all pgn files will be downloaded to
download_dir = "/home/benjafek/Downloads"
todays_date = datetime.now().strftime("%Y-%m-%d")
downloaded_name = os.path.join(download_dir, f"chess_com_games_{todays_date}.pgn")
output_dir = "/home/benjafek/chess/ben_chess/games"
my_user_name = "t1ber1us"

In [ ]:
def download():
    # First, download all the games
    path_to_driver = "/home/benjafek/chess/ben_chess/chromedriver/chromedriver"
    driver = webdriver.Chrome(path_to_driver)
    driver.get("https://www.chess.com/home")

    # Enter my email
    username = driver.find_element_by_name("_username")
    username.clear()
    username.send_keys("jafek91@gmail.com")

    # Enter my password <REDACTED>
    password = driver.find_element_by_name("_password")
    password.clear()
    password.send_keys("<REDACTED>")

    # Login
    login = driver.find_element_by_name("login")
    login.click()

    # Then find all my games
    for page_num in trange(1, 101):
        print (page_num)
        driver.get(f"https://www.chess.com/games/archive/{my_user_name}?gameType=live&gameTypeslive%5B%5D=bullet&page={page_num}")
        check_all = driver.find_element_by_class_name("archive-games-check-all")
        check_all.click()

        # Then click download
        download_button = driver.find_element_by_class_name("archive-games-download-button")
        download_button.click()
        time.sleep(1)

        # And move them to a unique name place
        new_name = os.path.join(output_dir, f"tiberius_games_page{page_num:05d}.pgn")
        os.rename(downloaded_name, new_name)
        time.sleep(0.5)
    driver.close()

In [ ]:
timezone_map = {
    "PDT": "US/Pacific",
    "EDT": "",
}

In [ ]:
# If the games aren't there, download them (download will take awhile)
all_games = sorted(glob.glob(os.path.join(output_dir, "*.pgn")))
if not all_games:
    download()

In [ ]:
# Now we can look at them
for game_list in all_games:
    with open(game_list, "r") as f:
        data = filter(None, f.read().strip().split('[Event "Live Chess"]\n'))

    for game in data:
        first_game = chess.pgn.read_game(io.StringIO(game))
        break
    break